In [67]:
import pandas as pd
import numpy as np

In [68]:
def unstack_data(df):
    rows = []
    rfids = [i for i in df.rfid.unique()]

    for rfid in rfids:
        temp = df[df.rfid==rfid]
        temp = temp.reset_index(drop=True)
        temp.sort_values(['trial_id'],inplace=True)
        temp.set_index('trial_id',inplace=True)
        temp_row = temp.iloc[:,1:].unstack().to_frame().sort_index(level=1).T
        temp_row.insert(loc=0, column='rfid', value=temp.rfid.unique())
        temp_row = temp_row.reset_index(drop=True)
        rows.append(temp_row)
        
    return rows


def deserialize_data(ts):
    if pd.isnull(ts):
        return []
    return [float(i) for i in ts.split()]

## subject

In [69]:
# the "index" table, all other tables should left join to it

In [70]:
sub = pd.read_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Raw/datastream_subject_table.csv')
#sub = sub[[i for i in sub.columns.tolist() if 'date' not in i]]
print(sub.shape)
sub.head()

(2381, 105)


,rfid,rat,cohort,experiment_group,drug_group,sex,arrival_date,age_at_arrival,uv,brevital,...,age_at_surgery,date_of_death,days_of_experiment,exit_day,last_good_session,exit_code,complete,tissue_collected,exit_notes,replaced_by
0,933000120124701,F1,1,Drug,Cocaine,Female,2017-06-29,NaN,NaN,NaN,...,NaN,2017-09-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,933000120124703,F2,1,Drug,Cocaine,Female,2017-06-29,NaN,NaN,NaN,...,NaN,2017-09-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,933000120117342,F3,1,Drug,Cocaine,Female,2017-06-29,NaN,NaN,NaN,...,NaN,2017-09-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,933000120117333,F4,1,Drug,Cocaine,Female,2017-06-29,NaN,NaN,NaN,...,NaN,2017-09-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,933000120117347,F5,1,Drug,Cocaine,Female,2017-06-29,NaN,NaN,NaN,...,NaN,2017-09-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
sub.to_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Unstacked/subject.csv')

## measurement

In [72]:
mmt = pd.read_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Raw/datastream_measurement_table.csv')
mmt.head()

,rfid,measurement_name,measurement_value,drug_group,cohort,measure_number,date_measured,technician
0,933000120124701,urine,NaN,cocaine,1,1,NaN,NaN
1,933000120124701,urine,NaN,cocaine,1,2,NaN,NaN
2,933000120124701,weight,177.0,cocaine,1,1,2017-07-24,NaN
3,933000120124701,weight,179.0,cocaine,1,2,2017-08-04,NaN
4,933000120124701,weight,190.0,cocaine,1,3,2017-08-11,NaN


In [73]:
mmt['measure_id'] = mmt['measurement_name'] + ['_']*len(mmt) + mmt['measure_number'].astype(str)
mmt_trimmed = mmt.drop(columns=['measurement_name','drug_group','cohort','measure_number','date_measured'])
mmt_trimmed.head()

,rfid,measurement_value,technician,measure_id
0,933000120124701,NaN,NaN,urine_1
1,933000120124701,NaN,NaN,urine_2
2,933000120124701,177.0,NaN,weight_1
3,933000120124701,179.0,NaN,weight_2
4,933000120124701,190.0,NaN,weight_3


In [74]:
rows = []
rfids = [i for i in mmt.rfid.unique()]

for rfid in rfids:
    temp = mmt_trimmed[mmt_trimmed.rfid == rfid]
    temp = temp.reset_index(drop=True)
    temp.sort_values(['measure_id'],inplace=True)
    temp.set_index('measure_id',inplace=True)
    temp_row = temp.iloc[:,1:].unstack().to_frame().sort_index(level=1).T
    temp_row.insert(loc=0, column='rfid', value=temp.rfid.unique())
    temp_row = temp_row.reset_index(drop=True)
    rows.append(temp_row)

In [75]:
mmt_unstacked = pd.concat(rows,ignore_index=True)
mmt_unstacked.columns = ['rfid'] + [i[0]+'_'+i[1] for i in mmt_unstacked.columns[1:]]
print(mmt_unstacked.shape)
mmt_unstacked.head()

(2381, 37)


,rfid,measurement_value_feces_1,technician_feces_1,measurement_value_feces_2,technician_feces_2,measurement_value_feces_3,technician_feces_3,measurement_value_feces_4,technician_feces_4,measurement_value_urine_1,...,measurement_value_weight_7,technician_weight_7,measurement_value_weight_8,technician_weight_8,measurement_value_weight_9,technician_weight_9,measurement_value_weight_11,technician_weight_11,measurement_value_weight_12,technician_weight_12
0,933000120124701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,933000120124703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,933000120117342,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,933000120117333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,933000120117347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
mmt_unstacked.to_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Unstacked/measurement_unstacked.csv')

## irritability

In [79]:
irr = pd.read_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Raw/irritability.csv')
irr.head()

,subject,rfid,cohort,sex,group,def_bsl_irr_1,agg_bsl_irr_1,irr_bsl_tot_1,irr_bsl_scorer_1,def_bsl_irr_2,...,def_drug_irr_3,agg_drug_irr_3,irr_drug_total_3,irr_drug_scorer_3,def_drug_ave,agg_drug_ave,total_drug_ave,diff_ave_def,diff_ave_agg,diff_ave_total
0,M58,933000120117320,1,M,cocaine,NaN,NaN,NaN,NaN,NaN,...,3.0,5.0,8.0,adam,4.666667,3.666667,8.333333,NaN,NaN,NaN
1,M59,933000120117325,1,M,cocaine,NaN,NaN,NaN,NaN,NaN,...,7.0,5.0,12.0,adam,8.666667,6.333333,15.000000,NaN,NaN,NaN
2,M61,933000120117307,1,M,cocaine,NaN,NaN,NaN,NaN,NaN,...,2.0,4.0,6.0,adam,6.666667,3.333333,10.000000,NaN,NaN,NaN
3,M62,933000120117322,1,M,cocaine,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M65,933000120117324,1,M,cocaine,NaN,NaN,NaN,NaN,NaN,...,13.0,1.0,14.0,adam,12.666667,0.666667,13.333333,NaN,NaN,NaN


In [80]:
irr_trimmed = irr.drop(columns=['subject', 'cohort', 'sex', 'group'])
print(irr_trimmed.shape)
irr_trimmed.head()

(662, 34)


,rfid,def_bsl_irr_1,agg_bsl_irr_1,irr_bsl_tot_1,irr_bsl_scorer_1,def_bsl_irr_2,agg_bsl_irr_2,irr_bsl_total_2,irr_bsl_scorer_2,def_bsl_irr_3,...,def_drug_irr_3,agg_drug_irr_3,irr_drug_total_3,irr_drug_scorer_3,def_drug_ave,agg_drug_ave,total_drug_ave,diff_ave_def,diff_ave_agg,diff_ave_total
0,933000120117320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,5.0,8.0,adam,4.666667,3.666667,8.333333,NaN,NaN,NaN
1,933000120117325,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,5.0,12.0,adam,8.666667,6.333333,15.000000,NaN,NaN,NaN
2,933000120117307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,4.0,6.0,adam,6.666667,3.333333,10.000000,NaN,NaN,NaN
3,933000120117322,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,933000120117324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13.0,1.0,14.0,adam,12.666667,0.666667,13.333333,NaN,NaN,NaN


In [81]:
irr_trimmed.to_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Unstacked/irritability.csv')

## tail immersion

In [82]:
ti = pd.read_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Raw/tail_immersion.csv')
ti.drop('#',axis=1,inplace=True)
ti.head()

,rfid,subject,cohort,sex,experiment_group,drug,tail_immersion_1_time,tail_immersion_2_time,tail_immersion_3_time,tail_immersion_difference_tolerance,tail_immersion_1_date,tail_immersion_2_date,tail_immersion_3_date
0,933000320045738,F301,3,Female,Drug,oxycodone,4.0,5.34,8.10,-2.76,2019-01-07,2019-01-21,2019-02-15
1,933000320045726,F302,3,Female,Drug,oxycodone,4.0,5.94,7.00,-1.06,2019-01-07,2019-01-21,2019-02-15
2,933000320045747,F303,3,Female,Drug,oxycodone,4.0,5.53,6.31,-0.78,2019-01-07,2019-01-21,2019-02-15
3,933000320045734,F304,3,Female,Drug,oxycodone,2.0,5.19,6.85,-1.66,2019-01-07,2019-01-21,2019-02-15
4,933000320045669,F305,3,Female,Drug,oxycodone,3.0,5.25,5.59,-0.34,2019-01-07,2019-01-21,2019-02-15


In [83]:
ti_trimmed = ti.drop(columns=['subject', 'cohort', 'sex', 'drug','tail_immersion_1_date',
                              'tail_immersion_2_date','tail_immersion_3_date','experiment_group'])
print(ti_trimmed.shape)
ti_trimmed.head()

(976, 5)


,rfid,tail_immersion_1_time,tail_immersion_2_time,tail_immersion_3_time,tail_immersion_difference_tolerance
0,933000320045738,4.0,5.34,8.10,-2.76
1,933000320045726,4.0,5.94,7.00,-1.06
2,933000320045747,4.0,5.53,6.31,-0.78
3,933000320045734,2.0,5.19,6.85,-1.66
4,933000320045669,3.0,5.25,5.59,-0.34


In [84]:
ti_trimmed.to_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Unstacked/tailimmersion.csv')

## voy frey

In [85]:
vf = pd.read_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Raw/von_frey.csv')
vf.drop('#',axis=1,inplace=True)
vf.head()

,rfid,subject,cohort,sex,experiment_group,drug,vf1_right_force_1,vf1_right_force_2,vf1_right_force_3,vf1_right_force_avg,...,vf2_left_time_1,vf2_left_time_2,vf2_left_time_3,vf2_left_avg,von_frey_2_date,von_frey_1_force,von_frey_1_time,von_frey_2_force,von_frey_2_time,von_frey_difference_force
0,933000320186887,M773,7,Male,Drug,oxycodone,31.9,8.0,26.9,22.3,...,NaN,NaN,NaN,NaN,2020-04-26,22.15,10.95,NaN,NaN,NaN
1,933000120138681,F101,1,Female,Drug,oxycodone,13.7,13.3,27.7,18.2,...,12.8,11.0,11.1,11.6,2018-08-30,23.40,13.70,24.55,12.20,-1.15
2,933000120138443,F102,1,Female,Drug,oxycodone,16.1,5.4,15.3,12.3,...,4.7,8.4,7.6,6.9,2018-08-30,13.65,16.00,13.05,6.40,0.60
3,933000120138699,F103,1,Female,Drug,oxycodone,27.9,24.7,12.9,21.8,...,6.8,3.9,2.8,4.5,2018-08-30,18.15,17.10,11.90,5.80,6.25
4,933000120138703,F104,1,Female,Drug,oxycodone,3.8,10.1,5.0,6.3,...,8.7,15.9,8.7,11.1,2018-08-30,8.55,12.35,19.90,9.85,-11.35


In [86]:
vf_trimmed = vf.drop(columns=['subject', 'cohort', 'sex', 'drug',
                              'von_frey_1_date', 'von_frey_2_date','experiment_group'])
print(vf_trimmed.shape)
vf_trimmed.head()

(1066, 38)


,rfid,vf1_right_force_1,vf1_right_force_2,vf1_right_force_3,vf1_right_force_avg,vf1_right_time_1,vf1_right_time_2,vf1_right_time_3,vf1_right_avg,vf1_left_force_1,...,vf2_left_force_avg,vf2_left_time_1,vf2_left_time_2,vf2_left_time_3,vf2_left_avg,von_frey_1_force,von_frey_1_time,von_frey_2_force,von_frey_2_time,von_frey_difference_force
0,933000320186887,31.9,8.0,26.9,22.3,15.9,3.8,13.4,11.0,15.5,...,NaN,NaN,NaN,NaN,NaN,22.15,10.95,NaN,NaN,NaN
1,933000120138681,13.7,13.3,27.7,18.2,6.7,6.5,13.8,9.0,30.3,...,23.4,12.8,11.0,11.1,11.6,23.40,13.70,24.55,12.20,-1.15
2,933000120138443,16.1,5.4,15.3,12.3,7.9,2.5,7.5,6.0,16.1,...,14.1,4.7,8.4,7.6,6.9,13.65,16.00,13.05,6.40,0.60
3,933000120138699,27.9,24.7,12.9,21.8,13.9,12.3,6.3,10.8,17.2,...,9.4,6.8,3.9,2.8,4.5,18.15,17.10,11.90,5.80,6.25
4,933000120138703,3.8,10.1,5.0,6.3,1.7,4.9,2.3,3.0,10.1,...,22.4,8.7,15.9,8.7,11.1,8.55,12.35,19.90,9.85,-11.35


In [87]:
vf_trimmed.to_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Unstacked/vonfrey_unstacked.csv')

## trial_note

In [88]:
note = pd.read_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Raw/trial_note.csv')
note.drop('#',axis=1,inplace=True)
note.head()

,rfid,subject,cohort,sex,drug,experiment_group,trial_id,start_date,code,to_do,note
0,933000320047006,M987,9,Male,Cocaine,Drug,LGA01,2019-10-08,Disconnect,Keep,NaN
1,933000320186832,M1072,10,Male,Cocaine,Drug,LGA02,2020-01-14,Died,Discard,NaN
2,933000320188172,M1565,15,Male,Cocaine,Drug,LGA01,2021-01-13,Tangled,Keep,NaN
3,933000320188113,M1553,15,Male,Cocaine,Drug,LGA06,2021-01-21,Tangled,Discard,NaN
4,933000320187895,M1572,15,Male,Cocaine,Drug,LGA11,2021-01-28,Sick/wound,Keep,NaN


In [89]:
note['drug'] = note['drug'].str.lower()
note['to_do'] = note['to_do'].apply(lambda x: x.lower().strip())

In [90]:
note_filter = note[note['to_do']=='discard'][['rfid','trial_id','drug','to_do']]
note_filter.reset_index(drop=True,inplace=True)

In [91]:
note_filter.replace('SHOCK','SHOCK_V3',inplace=True)
note_filter.replace('PRE-SHOCK','PRESHOCK',inplace=True)
note_filter.replace('TREAMENT02','TREATMENT02',inplace=True)
note_filter.replace('TREAMENT03','TREATMENT03',inplace=True)
note_filter.replace('PRE-SHOCK','PRESHOCK',inplace=True)

## trial_shock

In [92]:
shock = pd.read_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Raw/trial_shock.csv')
shock.head()

,rfid,subject,room,cohort,trial_id,drug,box,start_time,end_time,start_date,end_date,total_active_lever_presses,total_inactive_lever_presses,total_shocks,total_reward,rewards_after_first_shock,rewards_got_shock,reward_timestamps
0,933000320047293,M851,BSB273B,8,SHOCK_V3,cocaine,9,09:01:37,10:02:06,2019-08-02,2019-08-02,7,1,1,4,2,2,101 516 2079 2208
1,933000320047584,F907,BSB273B,9,SHOCK_V3,cocaine,7,09:16:37,10:18:30,2019-10-30,2019-10-30,3,4,1,3,0,3,1049 1243 1457
2,933000320187227,F1108,BSB273B,11,PRESHOCK,cocaine,8,08:12:49,09:28:39,2020-05-06,2020-05-06,3,4,1,3,0,3,74 96 394
3,933000320186956,M1151,BSB273B,11,SHOCK_V3,cocaine,9,07:35:00,08:44:14,2020-05-07,2020-05-07,4,0,2,3,1,2 3,158 487 1292
4,933000320048757,F1314,BSB273B,13,SHOCK_V3,cocaine,6,09:49:09,10:57:01,2020-09-09,2020-09-09,5,8,0,4,0,NaN,2137 2188 2976 3012


In [93]:
shock['rewards_got_shock'] = shock['rewards_got_shock'].apply(deserialize_data)
shock['reward_timestamps'] = shock['reward_timestamps'].apply(deserialize_data)

In [94]:
shock_trimmed = shock.drop(columns=['subject', 'room', 'cohort', 'box','start_date','end_date'])
print(shock_trimmed.shape)
shock_trimmed.head()

(2027, 12)


,rfid,trial_id,drug,start_time,end_time,total_active_lever_presses,total_inactive_lever_presses,total_shocks,total_reward,rewards_after_first_shock,rewards_got_shock,reward_timestamps
0,933000320047293,SHOCK_V3,cocaine,09:01:37,10:02:06,7,1,1,4,2,[2.0],"[101.0, 516.0, 2079.0, 2208.0]"
1,933000320047584,SHOCK_V3,cocaine,09:16:37,10:18:30,3,4,1,3,0,[3.0],"[1049.0, 1243.0, 1457.0]"
2,933000320187227,PRESHOCK,cocaine,08:12:49,09:28:39,3,4,1,3,0,[3.0],"[74.0, 96.0, 394.0]"
3,933000320186956,SHOCK_V3,cocaine,07:35:00,08:44:14,4,0,2,3,1,"[2.0, 3.0]","[158.0, 487.0, 1292.0]"
4,933000320048757,SHOCK_V3,cocaine,09:49:09,10:57:01,5,8,0,4,0,[],"[2137.0, 2188.0, 2976.0, 3012.0]"


In [95]:
shock_keep = pd.merge(shock_trimmed, note_filter, how='left', left_on=['rfid','trial_id','drug'], right_on=['rfid','trial_id','drug'])
shock_keep = shock_keep[~(shock_keep['to_do']=='discard')]
shock_keep.drop(columns=['drug','to_do'],inplace=True)
shock_keep.shape

(2021, 11)

In [96]:
shock_keep.isna().sum()

rfid                            0
trial_id                        0
start_time                      0
end_time                        0
total_active_lever_presses      0
total_inactive_lever_presses    0
total_shocks                    0
total_reward                    0
rewards_after_first_shock       0
rewards_got_shock               0
reward_timestamps               0
dtype: int64

In [97]:
rows = unstack_data(shock_keep)
shock_unstacked = pd.concat(rows,ignore_index=True)
shock_unstacked.columns = ['rfid'] + [i[0]+'_'+i[1] for i in shock_unstacked.columns[1:]]
print(shock_unstacked.shape)
shock_unstacked.head()

(891, 37)


,rfid,end_time_SHOCK_V3,reward_timestamps_SHOCK_V3,rewards_after_first_shock_SHOCK_V3,rewards_got_shock_SHOCK_V3,start_time_SHOCK_V3,total_active_lever_presses_SHOCK_V3,total_inactive_lever_presses_SHOCK_V3,total_reward_SHOCK_V3,total_shocks_SHOCK_V3,...,total_shocks_SHOCK_V1,end_time_SHOCK_V2,reward_timestamps_SHOCK_V2,rewards_after_first_shock_SHOCK_V2,rewards_got_shock_SHOCK_V2,start_time_SHOCK_V2,total_active_lever_presses_SHOCK_V2,total_inactive_lever_presses_SHOCK_V2,total_reward_SHOCK_V2,total_shocks_SHOCK_V2
0,933000320047293,10:02:06,"[101.0, 516.0, 2079.0, 2208.0]",2,[2.0],09:01:37,7,1,4,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,933000320047584,10:18:30,"[1049.0, 1243.0, 1457.0]",0,[3.0],09:16:37,3,4,3,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,933000320187227,08:44:14,"[9.0, 125.0]",0,[2.0],07:34:52,4,4,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,933000320186956,08:44:14,"[158.0, 487.0, 1292.0]",1,"[2.0, 3.0]",07:35:00,4,0,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,933000320048757,10:57:01,"[2137.0, 2188.0, 2976.0, 3012.0]",0,[],09:49:09,5,8,4,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
shock_unstacked = shock_unstacked[['rfid']+sorted(shock_unstacked.columns[1:])]
shock_unstacked.to_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Unstacked/shock_unstacked.csv')

## trial_pr

In [99]:
pr = pd.read_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Raw/trial_pr_copy.csv')
pr.head()

,rfid,subject,room,cohort,trial_id,drug,box,start_time,end_time,start_date,end_date,breakpoint,last_ratio,ratios,active_lever_presses,inactive_lever_presses,reward_presses
0,933000320047477,F801,BSB273B,8,PR01,cocaine,1.0,09:01:42,10:44:53,2019-07-10,2019-07-10,32.0,NaN,1 2 4 6 9 12 15 20 25 32,164,22,10
1,933000320187151,F1116,BSB273B,11,PR02,cocaine,16.0,08:55:13,13:40:45,2020-05-05,2020-05-05,178.0,NaN,1 2 4 6 9 12 15 20 25 32 40 50 62 77 95 118 14...,1021,137,18
2,933000320047673,F1010,BSB273B,15,TREATMENT02,cocaine,10.0,15:41:03,21:34:00,2020-11-13,2020-11-13,NaN,NaN,1 1 2 2 3 3 4 4 5 5 6 6 7 7 8 8 9 9 10 10 11 1...,216,5,26
3,933000320188630,F1715,BSB273B,17,PR01,cocaine,15.0,08:29:00,09:36:45,2021-06-18,2021-06-18,1.0,2.0,1,1,2,1
4,933000320047376,F935,BSB273C,9,PR03,cocaine,13.0,08:34:21,11:10:59,2019-11-01,2019-11-01,12.0,15.0,1 2 4 6 9 12,47,17,6


In [100]:
pr['ratios'] = pr['ratios'].apply(deserialize_data)

In [101]:
pr_trimmed = pr.drop(columns=['subject', 'room', 'cohort', 'box','start_date','end_date'])
print(pr_trimmed.shape)
pr_trimmed.head()

(6983, 11)


,rfid,trial_id,drug,start_time,end_time,breakpoint,last_ratio,ratios,active_lever_presses,inactive_lever_presses,reward_presses
0,933000320047477,PR01,cocaine,09:01:42,10:44:53,32.0,NaN,"[1.0, 2.0, 4.0, 6.0, 9.0, 12.0, 15.0, 20.0, 25...",164,22,10
1,933000320187151,PR02,cocaine,08:55:13,13:40:45,178.0,NaN,"[1.0, 2.0, 4.0, 6.0, 9.0, 12.0, 15.0, 20.0, 25...",1021,137,18
2,933000320047673,TREATMENT02,cocaine,15:41:03,21:34:00,NaN,NaN,"[1.0, 1.0, 2.0, 2.0, 3.0, 3.0, 4.0, 4.0, 5.0, ...",216,5,26
3,933000320188630,PR01,cocaine,08:29:00,09:36:45,1.0,2.0,[1.0],1,2,1
4,933000320047376,PR03,cocaine,08:34:21,11:10:59,12.0,15.0,"[1.0, 2.0, 4.0, 6.0, 9.0, 12.0]",47,17,6


In [102]:
pr_keep = pd.merge(pr_trimmed, note_filter, how='left', left_on=['rfid','trial_id','drug'], right_on=['rfid','trial_id','drug'])
pr_keep = pr_keep[~(pr_keep['to_do']=='discard')]
pr_keep.drop(columns=['drug','to_do'],inplace=True)
pr_keep.shape

(6907, 10)

In [103]:
pr_keep.isna().sum()

rfid                         0
trial_id                     0
start_time                   0
end_time                     0
breakpoint                 114
last_ratio                2814
ratios                       0
active_lever_presses         0
inactive_lever_presses       0
reward_presses               0
dtype: int64

In [104]:
rows = unstack_data(pr_keep)
pr_unstacked = pd.concat(rows,ignore_index=True)
pr_unstacked.columns = ['rfid'] + [i[0]+'_'+i[1] for i in pr_unstacked.columns[1:]]
print(pr_unstacked.shape)
pr_unstacked.head()

(1874, 57)


,rfid,active_lever_presses_PR01,breakpoint_PR01,end_time_PR01,inactive_lever_presses_PR01,last_ratio_PR01,ratios_PR01,reward_presses_PR01,start_time_PR01,active_lever_presses_PR02,...,reward_presses_TREATMENT01,start_time_TREATMENT01,active_lever_presses_TREATMENT04,breakpoint_TREATMENT04,end_time_TREATMENT04,inactive_lever_presses_TREATMENT04,last_ratio_TREATMENT04,ratios_TREATMENT04,reward_presses_TREATMENT04,start_time_TREATMENT04
0,933000320047477,164,32.0,10:44:53,22,NaN,"[1.0, 2.0, 4.0, 6.0, 9.0, 12.0, 15.0, 20.0, 25...",10,09:01:42,343,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,933000320187151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,933000320047673,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,933000320188630,1,1.0,09:36:45,2,2.0,[1.0],1,08:29:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,933000320047376,66,15.0,11:00:30,16,20.0,"[1.0, 2.0, 4.0, 6.0, 9.0, 12.0, 15.0]",7,08:57:51,416,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
pr_unstacked = pr_unstacked[['rfid']+sorted(pr_unstacked.columns[1:])]
pr_unstacked.to_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Unstacked/pr_unstacked.csv')

## trial_sha

In [136]:
sha = pd.read_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Raw/trial_sha.csv')
sha.head()

,rfid,subject,room,cohort,trial_id,drug,box,start_time,end_time,start_date,end_date,active_lever_presses,inactive_lever_presses,reward_presses,timeout_presses,active_timestamps,inactive_timestamps,reward_timestamps,timeout_timestamps
0,933000320047477,F801,BSB273B,8,SHA01,cocaine,1.0,08:52:23,10:53:23,2019-06-24,2019-06-24,17,4,10,7.0,22 88 90 94 97 172 175 224 239 606 616 4353 45...,466 639 3528 4880,22 88 172 224 606 4353 4519 5290 6786 7100,90 94 97 175 239 616 5293
1,933000320186957,F1106,BSB273B,11,SHA06,cocaine,6.0,07:43:38,10:41:02,2020-03-12,2020-03-12,0,0,0,0.0,NaN,NaN,NaN,NaN
2,933000320188356,F1611,BSB273B,16,SHA05,cocaine,11.0,08:37:57,11:03:35,2021-03-11,2021-03-11,13,3,6,7.0,101 103 110 110 150 3827 5671 6242 6244 6247 6...,6372 6382 6385,101 150 3827 5671 6242 6270,103 110 110 6244 6247 6279 6280
3,933000320047332,M952,BSB273C,9,SHA03,cocaine,16.0,08:06:05,10:43:42,2019-09-25,2019-09-25,11,10,8,3.0,6 56 274 340 628 1184 1184 2553 2564 3065 3067,361 364 553 1020 1021 1569 2571 2589 2589 6496,6 56 274 340 628 1184 2553 3065,1184 2564 3067
4,933000320048539,F1317,BSB273C,13,SHA10,cocaine,1.0,08:40:43,10:45:02,2020-08-13,2020-08-13,8,7,7,1.0,17 85 146 444 1164 1630 1634 3611,71 134 1172 2455 2463 2463 3671,17 85 146 444 1164 1630 3611,1634


In [137]:
sha['active_timestamps'] = sha['active_timestamps'].apply(deserialize_data)
sha['inactive_timestamps'] = sha['inactive_timestamps'].apply(deserialize_data)
sha['reward_timestamps'] = sha['reward_timestamps'].apply(deserialize_data)
sha['timeout_timestamps'] = sha['timeout_timestamps'].apply(deserialize_data)

In [138]:
sha_trimmed = sha.drop(columns=['subject', 'room', 'cohort', 'box','start_date','end_date'])
print(sha_trimmed.shape)
sha_trimmed.head()

(13499, 13)


,rfid,trial_id,drug,start_time,end_time,active_lever_presses,inactive_lever_presses,reward_presses,timeout_presses,active_timestamps,inactive_timestamps,reward_timestamps,timeout_timestamps
0,933000320047477,SHA01,cocaine,08:52:23,10:53:23,17,4,10,7.0,"[22.0, 88.0, 90.0, 94.0, 97.0, 172.0, 175.0, 2...","[466.0, 639.0, 3528.0, 4880.0]","[22.0, 88.0, 172.0, 224.0, 606.0, 4353.0, 4519...","[90.0, 94.0, 97.0, 175.0, 239.0, 616.0, 5293.0]"
1,933000320186957,SHA06,cocaine,07:43:38,10:41:02,0,0,0,0.0,[],[],[],[]
2,933000320188356,SHA05,cocaine,08:37:57,11:03:35,13,3,6,7.0,"[101.0, 103.0, 110.0, 110.0, 150.0, 3827.0, 56...","[6372.0, 6382.0, 6385.0]","[101.0, 150.0, 3827.0, 5671.0, 6242.0, 6270.0]","[103.0, 110.0, 110.0, 6244.0, 6247.0, 6279.0, ..."
3,933000320047332,SHA03,cocaine,08:06:05,10:43:42,11,10,8,3.0,"[6.0, 56.0, 274.0, 340.0, 628.0, 1184.0, 1184....","[361.0, 364.0, 553.0, 1020.0, 1021.0, 1569.0, ...","[6.0, 56.0, 274.0, 340.0, 628.0, 1184.0, 2553....","[1184.0, 2564.0, 3067.0]"
4,933000320048539,SHA10,cocaine,08:40:43,10:45:02,8,7,7,1.0,"[17.0, 85.0, 146.0, 444.0, 1164.0, 1630.0, 163...","[71.0, 134.0, 1172.0, 2455.0, 2463.0, 2463.0, ...","[17.0, 85.0, 146.0, 444.0, 1164.0, 1630.0, 361...",[1634.0]


In [139]:
sha_keep = pd.merge(sha_trimmed, note_filter, how='left', left_on=['rfid','trial_id','drug'], right_on=['rfid','trial_id','drug'])
sha_keep = sha_keep[~(sha_keep['to_do']=='discard')]
sha_keep.drop(columns=['drug','to_do'],inplace=True)
sha_keep.shape

(13468, 12)

In [140]:
sha_keep

,rfid,trial_id,start_time,end_time,active_lever_presses,inactive_lever_presses,reward_presses,timeout_presses,active_timestamps,inactive_timestamps,reward_timestamps,timeout_timestamps
0,933000320047477,SHA01,08:52:23,10:53:23,17,4,10,7.0,"[22.0, 88.0, 90.0, 94.0, 97.0, 172.0, 175.0, 2...","[466.0, 639.0, 3528.0, 4880.0]","[22.0, 88.0, 172.0, 224.0, 606.0, 4353.0, 4519...","[90.0, 94.0, 97.0, 175.0, 239.0, 616.0, 5293.0]"
1,933000320186957,SHA06,07:43:38,10:41:02,0,0,0,0.0,[],[],[],[]
2,933000320188356,SHA05,08:37:57,11:03:35,13,3,6,7.0,"[101.0, 103.0, 110.0, 110.0, 150.0, 3827.0, 56...","[6372.0, 6382.0, 6385.0]","[101.0, 150.0, 3827.0, 5671.0, 6242.0, 6270.0]","[103.0, 110.0, 110.0, 6244.0, 6247.0, 6279.0, ..."
3,933000320047332,SHA03,08:06:05,10:43:42,11,10,8,3.0,"[6.0, 56.0, 274.0, 340.0, 628.0, 1184.0, 1184....","[361.0, 364.0, 553.0, 1020.0, 1021.0, 1569.0, ...","[6.0, 56.0, 274.0, 340.0, 628.0, 1184.0, 2553....","[1184.0, 2564.0, 3067.0]"
4,933000320048539,SHA10,08:40:43,10:45:02,8,7,7,1.0,"[17.0, 85.0, 146.0, 444.0, 1164.0, 1630.0, 163...","[71.0, 134.0, 1172.0, 2455.0, 2463.0, 2463.0, ...","[17.0, 85.0, 146.0, 444.0, 1164.0, 1630.0, 361...",[1634.0]
...,...,...,...,...,...,...,...,...,...,...,...,...
13494,933000320525280,SHA03,15:29:42,18:38:13,7,2,5,2.0,"[4268.0, 4270.0, 4377.0, 4383.0, 4443.0, 5448....","[4441.0, 5063.0]","[4268.0, 4377.0, 4443.0, 5448.0, 5930.0]","[4270.0, 4383.0]"
13495,933000120117338,SHA03,00:00:00,00:00:00,3,7,3,NaN,"[478.98, 721.57, 4269.75]","[360.67, 661.27, 683.59, 4182.48, 6234.42, 624...","[478.98, 721.57, 4269.75]",[]
13496,933000120138344,SHA07,00:00:00,00:00:00,2,0,2,NaN,"[829.24, 5004.97]",[],"[829.24, 5004.97]",[]
13497,933000120138662,SHA04,00:00:00,00:00:00,2,0,1,1.0,"[3089.0, 3091.44]",[],[3089.0],[3091.44]


In [110]:
rows = unstack_data(sha_keep)
sha_unstacked = pd.concat(rows,ignore_index=True)
sha_unstacked.columns = ['rfid'] + [i[0]+'_'+i[1] for i in sha_unstacked.columns[1:]]
print(sha_unstacked.shape)
sha_unstacked.head()

(1848, 111)


,rfid,active_lever_presses_SHA01,active_timestamps_SHA01,end_time_SHA01,inactive_lever_presses_SHA01,inactive_timestamps_SHA01,reward_presses_SHA01,reward_timestamps_SHA01,start_time_SHA01,timeout_presses_SHA01,...,active_lever_presses_SHA11,active_timestamps_SHA11,end_time_SHA11,inactive_lever_presses_SHA11,inactive_timestamps_SHA11,reward_presses_SHA11,reward_timestamps_SHA11,start_time_SHA11,timeout_presses_SHA11,timeout_timestamps_SHA11
0,933000320047477,17,"[22.0, 88.0, 90.0, 94.0, 97.0, 172.0, 175.0, 2...",10:53:23,4,"[466.0, 639.0, 3528.0, 4880.0]",10,"[22.0, 88.0, 172.0, 224.0, 606.0, 4353.0, 4519...",08:52:23,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,933000320186957,0,[],11:37:44,0,[],0,[],15:12:50,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,933000320188356,0,[],10:53:46,0,[],0,[],08:53:11,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,933000320047332,17,"[101.0, 748.0, 1489.0, 1493.0, 1514.0, 1932.0,...",11:03:50,29,"[340.0, 555.0, 595.0, 770.0, 987.0, 991.0, 123...",8,"[101.0, 748.0, 1489.0, 1514.0, 1932.0, 3215.0,...",08:36:49,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,933000320048539,4,"[301.0, 2178.0, 2344.0, 2345.0]",11:07:17,0,[],3,"[301.0, 2178.0, 2344.0]",09:06:51,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [111]:
sha_unstacked = sha_unstacked[['rfid']+sorted(sha_unstacked.columns[1:])]
sha_unstacked.to_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Unstacked/sha_unstacked.csv')

## trial_lga

In [112]:
lga = pd.read_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Raw/trial_lga.csv')
lga['timeout_presses'] = lga['active_lever_presses'] - lga['reward_presses']
lga.head()

/var/folders/6r/ljk6dgwn77q618x071444l0r0000gn/T/ipykernel_10783/2780113061.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  lga = pd.read_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Raw/trial_lga.csv')


,rfid,subject,room,cohort,trial_id,drug,box,start_time,end_time,start_date,end_date,active_lever_presses,inactive_lever_presses,reward_presses,timeout_presses,active_timestamps,inactive_timestamps,reward_timestamps,timeout_timestamps
0,933000320047584,F907,BSB273B,9,LGA16,cocaine,7.0,08:32:26,14:33:10,2019-11-05,2019-11-05,80,0,62,18,489 8361 8379 9149 9159 9165 9171 9172 9176 91...,NaN,489 8361 9149 9171 9194 9215 9239 9262 10125 1...,8379 9159 9165 9172 9176 9183 9188 10125 10158...
1,933000320046920,M857,BSB273D,8,LGA18,cocaine,5.0,08:53:04,15:23:30,2019-08-12,2019-08-12,36,3,34,2,2 31 763 1743 2276 3344 12176 12297 12397 1373...,16835 19781 20855,2 31 763 1743 2276 3344 12176 12297 12397 1373...,19508 20328
2,933000320047040,M955,BSB273D,9,LGA16,cocaine,3.0,07:58:14,14:34:45,2019-11-05,2019-11-05,130,0,125,5,10 33 54 83 127 227 312 366 458 752 915 996 10...,NaN,10 33 54 83 127 227 312 366 458 752 915 996 10...,10685 11871 12517 14546 21060
3,933000320187895,M1572,BSB273E,15,LGA16,cocaine,4.0,08:10:07,14:35:14,2021-02-10,2021-02-10,115,1,95,20,7 7 88 127 233 396 524 651 836 1000 1185 1413 ...,82,7 88 127 233 396 524 651 836 1000 1185 1413 15...,7 3474 3474 4083 4496 5205 5420 5706 5706 6458...
4,933000320499961,F1828,MTF134C,18,LGA03,cocaine,12.0,09:18:59,15:19:22,2021-09-17,2021-09-17,107,1,99,8,27 214 220 264 585 844 2426 2513 2800 2975 324...,41,27 214 264 585 844 2426 2513 2800 2975 3243 33...,220 3632 16915 18333 18853 19167 19167 19167


In [113]:
lga['active_timestamps'] = lga['active_timestamps'].apply(deserialize_data)
lga['inactive_timestamps'] = lga['inactive_timestamps'].apply(deserialize_data)
lga['reward_timestamps'] = lga['reward_timestamps'].apply(deserialize_data)
lga['timeout_timestamps'] = lga['timeout_timestamps'].apply(deserialize_data)

In [114]:
lga_trimmed = lga.drop(columns=['subject', 'room', 'cohort', 'box','start_date','end_date'])
print(lga_trimmed.shape)
lga_trimmed.head()

(33020, 13)


,rfid,trial_id,drug,start_time,end_time,active_lever_presses,inactive_lever_presses,reward_presses,timeout_presses,active_timestamps,inactive_timestamps,reward_timestamps,timeout_timestamps
0,933000320047584,LGA16,cocaine,08:32:26,14:33:10,80,0,62,18,"[489.0, 8361.0, 8379.0, 9149.0, 9159.0, 9165.0...",[],"[489.0, 8361.0, 9149.0, 9171.0, 9194.0, 9215.0...","[8379.0, 9159.0, 9165.0, 9172.0, 9176.0, 9183...."
1,933000320046920,LGA18,cocaine,08:53:04,15:23:30,36,3,34,2,"[2.0, 31.0, 763.0, 1743.0, 2276.0, 3344.0, 121...","[16835.0, 19781.0, 20855.0]","[2.0, 31.0, 763.0, 1743.0, 2276.0, 3344.0, 121...","[19508.0, 20328.0]"
2,933000320047040,LGA16,cocaine,07:58:14,14:34:45,130,0,125,5,"[10.0, 33.0, 54.0, 83.0, 127.0, 227.0, 312.0, ...",[],"[10.0, 33.0, 54.0, 83.0, 127.0, 227.0, 312.0, ...","[10685.0, 11871.0, 12517.0, 14546.0, 21060.0]"
3,933000320187895,LGA16,cocaine,08:10:07,14:35:14,115,1,95,20,"[7.0, 7.0, 88.0, 127.0, 233.0, 396.0, 524.0, 6...",[82.0],"[7.0, 88.0, 127.0, 233.0, 396.0, 524.0, 651.0,...","[7.0, 3474.0, 3474.0, 4083.0, 4496.0, 5205.0, ..."
4,933000320499961,LGA03,cocaine,09:18:59,15:19:22,107,1,99,8,"[27.0, 214.0, 220.0, 264.0, 585.0, 844.0, 2426...",[41.0],"[27.0, 214.0, 264.0, 585.0, 844.0, 2426.0, 251...","[220.0, 3632.0, 16915.0, 18333.0, 18853.0, 191..."


In [115]:
lga_keep = pd.merge(lga_trimmed, note_filter, how='left', left_on=['rfid','trial_id','drug'], right_on=['rfid','trial_id','drug'])
lga_keep = lga_keep[~(lga_keep['to_do']=='discard')]
lga_keep.drop(columns=['drug','to_do'],inplace=True)
lga_keep.shape

(32453, 12)

In [116]:
lga_keep.isna().sum()

rfid                      0
trial_id                  0
start_time                0
end_time                  0
active_lever_presses      0
inactive_lever_presses    0
reward_presses            0
timeout_presses           0
active_timestamps         0
inactive_timestamps       0
reward_timestamps         0
timeout_timestamps        0
dtype: int64

In [117]:
rows = unstack_data(lga_keep)
lga_unstacked = pd.concat(rows,ignore_index=True)
lga_unstacked.columns = ['rfid'] + [i[0]+'_'+i[1] for i in lga_unstacked.columns[1:]]
print(lga_unstacked.shape)
lga_unstacked.head()

(1857, 281)


,rfid,active_lever_presses_LGA01,active_timestamps_LGA01,end_time_LGA01,inactive_lever_presses_LGA01,inactive_timestamps_LGA01,reward_presses_LGA01,reward_timestamps_LGA01,start_time_LGA01,timeout_presses_LGA01,...,active_lever_presses_LGA28,active_timestamps_LGA28,end_time_LGA28,inactive_lever_presses_LGA28,inactive_timestamps_LGA28,reward_presses_LGA28,reward_timestamps_LGA28,start_time_LGA28,timeout_presses_LGA28,timeout_timestamps_LGA28
0,933000320047584,6,"[12842.0, 14655.0, 16795.0, 16907.0, 16920.0, ...",14:54:58,0,[],4,"[12842.0, 14655.0, 16795.0, 16907.0]",08:06:56,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,933000320046920,3,"[11207.0, 13256.0, 15685.0]",15:40:51,5,"[11264.0, 15157.0, 16388.0, 19071.0, 19073.0]",3,"[11207.0, 13256.0, 15685.0]",09:40:12,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,933000320047040,178,"[96.0, 99.0, 159.0, 892.0, 1208.0, 1618.0, 186...",15:01:58,2,"[108.0, 19678.0]",115,"[96.0, 159.0, 892.0, 1208.0, 1618.0, 1861.0, 2...",09:01:08,63,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,933000320187895,147,"[0.0, 23.0, 24.0, 28.0, 53.0, 92.0, 106.0, 116...",14:57:02,9,"[142.0, 241.0, 499.0, 512.0, 516.0, 521.0, 315...",89,"[0.0, 23.0, 53.0, 92.0, 116.0, 230.0, 349.0, 4...",08:10:24,58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,933000320499961,169,"[123.0, 128.0, 168.0, 358.0, 548.0, 607.0, 869...",15:32:58,0,[],101,"[123.0, 168.0, 358.0, 548.0, 607.0, 869.0, 117...",09:00:20,68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
lga_unstacked = lga_unstacked[['rfid']+sorted(lga_unstacked.columns[1:])]
lga_unstacked.to_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Unstacked/lga_unstacked.csv')

# JOINING ALL TABLES HERE 

#### (9 tables in total, Notes table excluded since it has been merged to each of the trial data tables)

In [119]:
sub.shape

(2381, 105)

In [120]:
mmt_unstacked.shape

(2381, 37)

In [121]:
irr_trimmed.shape

(662, 34)

In [122]:
ti_trimmed.shape

(976, 5)

In [123]:
vf_trimmed.shape

(1066, 38)

In [124]:
shock_unstacked.shape

(891, 37)

In [125]:
pr_unstacked.shape

(1874, 57)

In [126]:
sha_unstacked.shape

(1848, 111)

In [127]:
lga_unstacked.shape

(1857, 281)

In [128]:
test = (
    sub.merge(mmt_unstacked, how='left', on='rfid')
        .merge(irr_trimmed, how='left', on='rfid')
        .merge(ti_trimmed, how='left', on='rfid')
        .merge(vf_trimmed, how='left', on='rfid')
        .merge(shock_unstacked, how='left', on='rfid')
        .merge(pr_unstacked, how='left', on='rfid')
        .merge(sha_unstacked, how='left', on='rfid')
        .merge(lga_unstacked, how='left', on='rfid')
)
test

,rfid,rat,cohort,experiment_group,drug_group,sex,arrival_date,age_at_arrival,uv,brevital,...,timeout_timestamps_LGA19,timeout_timestamps_LGA20,timeout_timestamps_LGA21,timeout_timestamps_LGA22,timeout_timestamps_LGA23,timeout_timestamps_LGA24,timeout_timestamps_LGA25,timeout_timestamps_LGA26,timeout_timestamps_LGA27,timeout_timestamps_LGA28
0,933000120124701,F1,1,Drug,Cocaine,Female,2017-06-29,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,933000120124703,F2,1,Drug,Cocaine,Female,2017-06-29,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,933000120117342,F3,1,Drug,Cocaine,Female,2017-06-29,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,933000120117333,F4,1,Drug,Cocaine,Female,2017-06-29,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,933000120117347,F5,1,Drug,Cocaine,Female,2017-06-29,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2376,933000320625139,M1883,18,Naive,Oxycodone,Male,2022-08-31,36.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2377,933000320624877,M1884,18,Naive,Oxycodone,Male,2022-08-31,36.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2378,933000320625141,M1885,18,Naive,Oxycodone,Male,2022-08-31,36.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2379,933000320624891,M1886,18,Naive,Oxycodone,Male,2022-08-31,36.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [129]:
test.shape

(2381, 697)

In [130]:
ordered_cols = [
    'rfid',
    'rat',
    'cohort',
    'sex',
    'drug_group',
    'experiment_group',
    'dissection_group',
    'brevital',
    'last_good_session',
    'exit_code',
    'complete',
    'tissue_collected', 
    'exit_notes',
    'replaced_by',
    'coat_color',
    'ear_punch',
    'litter_number',
    'litter_size',
    'rack',
    'shipping_box',
    'uv',
    'female_swab_1_analysis',
    'female_swab_2_analysis',
    'female_swab_3_analysis',
    'reward_presses_SHA01',
    'reward_presses_SHA02',
    'reward_presses_SHA03',
    'reward_presses_SHA04',
    'reward_presses_SHA05',
    'reward_presses_SHA06',
    'reward_presses_SHA07',
    'reward_presses_SHA08',
    'reward_presses_SHA09',
    'reward_presses_SHA10',
    'active_lever_presses_SHA01',
    'active_lever_presses_SHA02',
    'active_lever_presses_SHA03',
    'active_lever_presses_SHA04',
    'active_lever_presses_SHA05',
    'active_lever_presses_SHA06',
    'active_lever_presses_SHA07',
    'active_lever_presses_SHA08',
    'active_lever_presses_SHA09',
    'active_lever_presses_SHA10',
    'timeout_presses_SHA01',
    'timeout_presses_SHA02', 
    'timeout_presses_SHA03',
    'timeout_presses_SHA04',
    'timeout_presses_SHA05',
    'timeout_presses_SHA06',
    'timeout_presses_SHA07',
    'timeout_presses_SHA08',
    'timeout_presses_SHA09',
    'timeout_presses_SHA10',
    'inactive_lever_presses_SHA01',
    'inactive_lever_presses_SHA02',
    'inactive_lever_presses_SHA03',
    'inactive_lever_presses_SHA04',
    'inactive_lever_presses_SHA05',
    'inactive_lever_presses_SHA06',
    'inactive_lever_presses_SHA07',
    'inactive_lever_presses_SHA08',
    'inactive_lever_presses_SHA09',
    'inactive_lever_presses_SHA10',
    'reward_presses_LGA01',
    'reward_presses_LGA02',
    'reward_presses_LGA03',
    'reward_presses_LGA04',
    'reward_presses_LGA05',
    'reward_presses_LGA06',
    'reward_presses_LGA07',
    'reward_presses_LGA08',
    'reward_presses_LGA09',
    'reward_presses_LGA10',
    'reward_presses_LGA11',
    'reward_presses_LGA12',
    'reward_presses_LGA13',
    'reward_presses_LGA14',
    'active_lever_presses_LGA01',
    'active_lever_presses_LGA02',
    'active_lever_presses_LGA03',
    'active_lever_presses_LGA04',
    'active_lever_presses_LGA05',
    'active_lever_presses_LGA06',
    'active_lever_presses_LGA07',
    'active_lever_presses_LGA08',
    'active_lever_presses_LGA09',
    'active_lever_presses_LGA10',
    'active_lever_presses_LGA11',
    'active_lever_presses_LGA12',
    'active_lever_presses_LGA13',
    'active_lever_presses_LGA14',
    'timeout_presses_LGA01',
    'timeout_presses_LGA02',
    'timeout_presses_LGA03',
    'timeout_presses_LGA04',
    'timeout_presses_LGA05',
    'timeout_presses_LGA06',
    'timeout_presses_LGA07',
    'timeout_presses_LGA08',
    'timeout_presses_LGA09',
    'timeout_presses_LGA10',
    'timeout_presses_LGA11',
    'timeout_presses_LGA12',
    'timeout_presses_LGA13',
    'timeout_presses_LGA14',
    'inactive_lever_presses_LGA01',
    'inactive_lever_presses_LGA02',
    'inactive_lever_presses_LGA03',
    'inactive_lever_presses_LGA04',
    'inactive_lever_presses_LGA05',
    'inactive_lever_presses_LGA06',
    'inactive_lever_presses_LGA07',
    'inactive_lever_presses_LGA08',
    'inactive_lever_presses_LGA09',
    'inactive_lever_presses_LGA10',
    'inactive_lever_presses_LGA11',
    'inactive_lever_presses_LGA12',
    'inactive_lever_presses_LGA13',
    'inactive_lever_presses_LGA14',
    'reward_presses_PR01',
    'reward_presses_PR02',
    'reward_presses_PR03',
    'active_lever_presses_PR01',
    'active_lever_presses_PR02',
    'active_lever_presses_PR03',
    'inactive_lever_presses_PR01',
    'inactive_lever_presses_PR02',
    'inactive_lever_presses_PR03',
    'breakpoint_PR01',
    'breakpoint_PR02',
    'breakpoint_PR03',
    'last_ratio_PR01',
    'last_ratio_PR02',
    'last_ratio_PR03',
    'treatment_1_group',
    'treatment_2_group',
    'treatment_3_group',
    'treatment_4_group',
    'reward_presses_TREATMENT01',
    'reward_presses_TREATMENT02',
    'reward_presses_TREATMENT03',
    'reward_presses_TREATMENT04',
    'active_lever_presses_TREATMENT01',
    'active_lever_presses_TREATMENT02',
    'active_lever_presses_TREATMENT03',
    'active_lever_presses_TREATMENT04',
    'inactive_lever_presses_TREATMENT01',
    'inactive_lever_presses_TREATMENT02',
    'inactive_lever_presses_TREATMENT03',
    'inactive_lever_presses_TREATMENT04',
    'breakpoint_TREATMENT01',
    'breakpoint_TREATMENT02',
    'breakpoint_TREATMENT03',
    'breakpoint_TREATMENT04',
    'last_ratio_TREATMENT01',
    'last_ratio_TREATMENT02',
    'last_ratio_TREATMENT03',
    'last_ratio_TREATMENT04',
    'total_reward_PRESHOCK',
    'total_reward_SHOCK_V1',
    'total_reward_SHOCK_V2',
    'total_reward_SHOCK_V3',
    'total_active_lever_presses_PRESHOCK',
    'total_active_lever_presses_SHOCK_V1',
    'total_active_lever_presses_SHOCK_V2',
    'total_active_lever_presses_SHOCK_V3',
    'total_inactive_lever_presses_PRESHOCK',
    'total_inactive_lever_presses_SHOCK_V1',
    'total_inactive_lever_presses_SHOCK_V2',
    'total_inactive_lever_presses_SHOCK_V3',
    'total_shocks_PRESHOCK',
    'total_shocks_SHOCK_V1',
    'total_shocks_SHOCK_V2',
    'total_shocks_SHOCK_V3',
    'rewards_after_first_shock_PRESHOCK',
    'rewards_after_first_shock_SHOCK_V1',
    'rewards_after_first_shock_SHOCK_V2',
    'rewards_after_first_shock_SHOCK_V3',
    'reward_presses_SHA11',
    'active_lever_presses_SHA11',
    'timeout_presses_SHA11',
    'inactive_lever_presses_SHA11',
    'reward_presses_LGA15',
    'reward_presses_LGA16',
    'reward_presses_LGA17',
    'reward_presses_LGA18',
    'reward_presses_LGA19',
    'reward_presses_LGA20',
    'reward_presses_LGA21',
    'reward_presses_LGA22',
    'reward_presses_LGA23',
    'reward_presses_LGA24',
    'reward_presses_LGA25',
    'reward_presses_LGA26',
    'reward_presses_LGA27',
    'reward_presses_LGA28',
    'active_lever_presses_LGA15',
    'active_lever_presses_LGA16',
    'active_lever_presses_LGA17',
    'active_lever_presses_LGA18',
    'active_lever_presses_LGA19',
    'active_lever_presses_LGA20',
    'active_lever_presses_LGA21',
    'active_lever_presses_LGA22',
    'active_lever_presses_LGA23',
    'active_lever_presses_LGA24',
    'active_lever_presses_LGA25',
    'active_lever_presses_LGA26',
    'active_lever_presses_LGA27',
    'active_lever_presses_LGA28',
    'timeout_presses_LGA15',
    'timeout_presses_LGA16',
    'timeout_presses_LGA17',
    'timeout_presses_LGA18',
    'timeout_presses_LGA19',
    'timeout_presses_LGA20',
    'timeout_presses_LGA21',
    'timeout_presses_LGA22',
    'timeout_presses_LGA23',
    'timeout_presses_LGA24',
    'timeout_presses_LGA25',
    'timeout_presses_LGA26',
    'timeout_presses_LGA27',
    'timeout_presses_LGA28',
    'inactive_lever_presses_LGA15',
    'inactive_lever_presses_LGA16',
    'inactive_lever_presses_LGA17',
    'inactive_lever_presses_LGA18',
    'inactive_lever_presses_LGA19',
    'inactive_lever_presses_LGA20',
    'inactive_lever_presses_LGA21',
    'inactive_lever_presses_LGA22',
    'inactive_lever_presses_LGA23',
    'inactive_lever_presses_LGA24',
    'inactive_lever_presses_LGA25',
    'inactive_lever_presses_LGA26',
    'inactive_lever_presses_LGA27',
    'inactive_lever_presses_LGA28',
    'von_frey_1_force',
    'von_frey_1_time',
    'von_frey_2_force',
    'von_frey_2_time',
    'von_frey_difference_force',
    'tail_immersion_1_time',
    'tail_immersion_2_time',
    'tail_immersion_3_time',
    'tail_immersion_difference_tolerance',
    'def_bsl_ave',
    'agg_bsl_ave',
    'total_bsl_ave',
    'def_drug_ave',
    'agg_drug_ave',
    'total_drug_ave',
    'diff_ave_def',
    'diff_ave_agg',
    'diff_ave_total',
    'reward_timestamps_SHA01',
    'reward_timestamps_SHA02',
    'reward_timestamps_SHA03',
    'reward_timestamps_SHA04',
    'reward_timestamps_SHA05',
    'reward_timestamps_SHA06',
    'reward_timestamps_SHA07',
    'reward_timestamps_SHA08',
    'reward_timestamps_SHA09',
    'reward_timestamps_SHA10',
    'active_timestamps_SHA01',
    'active_timestamps_SHA02',
    'active_timestamps_SHA03',
    'active_timestamps_SHA04',
    'active_timestamps_SHA05',
    'active_timestamps_SHA06',
    'active_timestamps_SHA07',
    'active_timestamps_SHA08',
    'active_timestamps_SHA09',
    'active_timestamps_SHA10',
    'timeout_timestamps_SHA01',
    'timeout_timestamps_SHA02',
    'timeout_timestamps_SHA03',
    'timeout_timestamps_SHA04',
    'timeout_timestamps_SHA05',
    'timeout_timestamps_SHA06',
    'timeout_timestamps_SHA07',
    'timeout_timestamps_SHA08',
    'timeout_timestamps_SHA09',
    'timeout_timestamps_SHA10',
    'inactive_timestamps_SHA01',
    'inactive_timestamps_SHA02',
    'inactive_timestamps_SHA03',
    'inactive_timestamps_SHA04',
    'inactive_timestamps_SHA05',
    'inactive_timestamps_SHA06',
    'inactive_timestamps_SHA07',
    'inactive_timestamps_SHA08',
    'inactive_timestamps_SHA09',
    'inactive_timestamps_SHA10',
    'reward_timestamps_LGA01',
    'reward_timestamps_LGA02',
    'reward_timestamps_LGA03',
    'reward_timestamps_LGA04',
    'reward_timestamps_LGA05',
    'reward_timestamps_LGA06',
    'reward_timestamps_LGA07',
    'reward_timestamps_LGA08',
    'reward_timestamps_LGA09',
    'reward_timestamps_LGA10',
    'reward_timestamps_LGA11',
    'reward_timestamps_LGA12',
    'reward_timestamps_LGA13',
    'reward_timestamps_LGA14',
    'active_timestamps_LGA01',
    'active_timestamps_LGA02',
    'active_timestamps_LGA03',
    'active_timestamps_LGA04',
    'active_timestamps_LGA05',
    'active_timestamps_LGA06',
    'active_timestamps_LGA07',
    'active_timestamps_LGA08',
    'active_timestamps_LGA09',
    'active_timestamps_LGA10',
    'active_timestamps_LGA11',
    'active_timestamps_LGA12',
    'active_timestamps_LGA13',
    'active_timestamps_LGA14',
    'timeout_timestamps_LGA01',
    'timeout_timestamps_LGA02',
    'timeout_timestamps_LGA03',
    'timeout_timestamps_LGA04',
    'timeout_timestamps_LGA05',
    'timeout_timestamps_LGA06',
    'timeout_timestamps_LGA07',
    'timeout_timestamps_LGA08',
    'timeout_timestamps_LGA09',
    'timeout_timestamps_LGA10',
    'timeout_timestamps_LGA11',
    'timeout_timestamps_LGA12',
    'timeout_timestamps_LGA13',
    'timeout_timestamps_LGA14',
    'inactive_timestamps_LGA01',
    'inactive_timestamps_LGA02',
    'inactive_timestamps_LGA03',
    'inactive_timestamps_LGA04',
    'inactive_timestamps_LGA05',
    'inactive_timestamps_LGA06',
    'inactive_timestamps_LGA07',
    'inactive_timestamps_LGA08',
    'inactive_timestamps_LGA09',
    'inactive_timestamps_LGA10',
    'inactive_timestamps_LGA11',
    'inactive_timestamps_LGA12',
    'inactive_timestamps_LGA13',
    'inactive_timestamps_LGA14',
    'ratios_PR01',
    'ratios_PR02',
    'ratios_PR03',
    'ratios_TREATMENT01',
    'ratios_TREATMENT02',
    'ratios_TREATMENT03',
    'ratios_TREATMENT04',
    'group_pre_shock',
    'group_shock',
    'rewards_got_shock_PRESHOCK',
    'rewards_got_shock_SHOCK_V1',
    'rewards_got_shock_SHOCK_V2',
    'rewards_got_shock_SHOCK_V3',
    'reward_timestamps_PRESHOCK',
    'reward_timestamps_SHOCK_V1',
    'reward_timestamps_SHOCK_V2',
    'reward_timestamps_SHOCK_V3',
    'reward_timestamps_SHA11',
    'active_timestamps_SHA11',
    'timeout_timestamps_SHA11',
    'inactive_timestamps_SHA11',
    'reward_timestamps_LGA15',
    'reward_timestamps_LGA16',
    'reward_timestamps_LGA17',
    'reward_timestamps_LGA18',
    'reward_timestamps_LGA19',
    'reward_timestamps_LGA20',
    'reward_timestamps_LGA21',
    'reward_timestamps_LGA22',
    'reward_timestamps_LGA23',
    'reward_timestamps_LGA24',
    'reward_timestamps_LGA25',
    'reward_timestamps_LGA26',
    'reward_timestamps_LGA27',
    'reward_timestamps_LGA28',
    'active_timestamps_LGA15',
    'active_timestamps_LGA16',
    'active_timestamps_LGA17',
    'active_timestamps_LGA18',
    'active_timestamps_LGA19',
    'active_timestamps_LGA20',
    'active_timestamps_LGA21',
    'active_timestamps_LGA22',
    'active_timestamps_LGA23',
    'active_timestamps_LGA24',
    'active_timestamps_LGA25',
    'active_timestamps_LGA26',
    'active_timestamps_LGA27',
    'active_timestamps_LGA28',
    'timeout_timestamps_LGA15',
    'timeout_timestamps_LGA16',
    'timeout_timestamps_LGA17',
    'timeout_timestamps_LGA18',
    'timeout_timestamps_LGA19',
    'timeout_timestamps_LGA20',
    'timeout_timestamps_LGA21',
    'timeout_timestamps_LGA22',
    'timeout_timestamps_LGA23',
    'timeout_timestamps_LGA24',
    'timeout_timestamps_LGA25',
    'timeout_timestamps_LGA26',
    'timeout_timestamps_LGA27',
    'timeout_timestamps_LGA28',
    'inactive_timestamps_LGA15',
    'inactive_timestamps_LGA16',
    'inactive_timestamps_LGA17',
    'inactive_timestamps_LGA18',
    'inactive_timestamps_LGA19',
    'inactive_timestamps_LGA20',
    'inactive_timestamps_LGA21',
    'inactive_timestamps_LGA22',
    'inactive_timestamps_LGA23',
    'inactive_timestamps_LGA24',
    'inactive_timestamps_LGA25',
    'inactive_timestamps_LGA26',
    'inactive_timestamps_LGA27',
    'inactive_timestamps_LGA28',
    'vf1_right_force_1',
    'vf1_right_force_2',
    'vf1_right_force_3',
    'vf1_right_force_avg',
    'vf1_right_time_1',
    'vf1_right_time_2',
    'vf1_right_time_3',
    'vf1_right_avg',
    'vf1_left_force_1',
    'vf1_left_force_2',
    'vf1_left_force_3',
    'vf1_left_force_avg',
    'vf1_left_time_1',
    'vf1_left_time_2',
    'vf1_left_time_3',
    'vf1_left_avg',
    'vf2_right_force_1',
    'vf2_right_force_2',
    'vf2_right_force_3',
    'vf2_right_force_avg',
    'vf2_right_time_1',
    'vf2_right_time_2',
    'vf2_right_time_3',
    'vf2_right_avg',
    'vf2_left_force_1',
    'vf2_left_force_2',
    'vf2_left_force_3',
    'vf2_left_force_avg',
    'vf2_left_time_1',
    'vf2_left_time_2',
    'vf2_left_time_3',
    'vf2_left_avg',
    'def_bsl_irr_1',
    'agg_bsl_irr_1',
    'irr_bsl_tot_1',
    'def_bsl_irr_2',
    'agg_bsl_irr_2',
    'irr_bsl_total_2',
    'def_bsl_irr_3',
    'agg_bsl_irr_3',
    'irr_bsl_total_3',
    'def_drug_irr_1',
    'agg_drug_irr_1',
    'irr_drug_tot_1',
    'def_drug_irr_2',
    'agg_drug_irr_2',
    'irr_drug_total_2',
    'def_drug_irr_3',
    'agg_drug_irr_3',
    'irr_drug_total_3',
    'measurement_value_feces_1',
    'measurement_value_feces_2',
    'measurement_value_feces_3',
    'measurement_value_feces_4',
    'measurement_value_urine_1',
    'measurement_value_urine_2',
    'measurement_value_weight_1',
    'measurement_value_weight_2',
    'measurement_value_weight_3',
    'measurement_value_weight_4',
    'measurement_value_weight_5',
    'measurement_value_weight_6',
    'measurement_value_weight_7',
    'measurement_value_weight_8',
    'measurement_value_weight_9',
    'measurement_value_weight_10',
    'measurement_value_weight_11',
    'measurement_value_weight_12',
    'days_of_experiment',
    'age_at_arrival',
    'age_at_surgery',
    'age_at_sha',
    'age_at_lga',
    'age_at_dissection',
    'dissection_date',
    'date_of_death',
    'exit_day',
    'date_of_birth',
    'date_of_wean',
    'date_of_ship',
    'arrival_date',
    'date_of_eye_bleed',
    'surgery_date',
    'recatheter_surgery_date',
    'short_access_start_date',
    'short_access_end_date',
    'progressive_ratio_1_date',
    'long_access_start_date',
    'long_access_end_date',
    'progressive_ratio_2_date',
    'pre_shock_date',
    'shock_1_date',
    'shock_2_date',
    'shock_3_date',
    'progressive_ratio_3_date',
    'brevital_date',
    'start_time_SHA01',
    'end_time_SHA01',
    'start_time_SHA02',
    'end_time_SHA02',
    'start_time_SHA03',
    'end_time_SHA03',
    'start_time_SHA04',
    'end_time_SHA04',
    'start_time_SHA05',
    'end_time_SHA05',
    'start_time_SHA06',
    'end_time_SHA06',
    'start_time_SHA07',
    'end_time_SHA07',
    'start_time_SHA08',
    'end_time_SHA08',
    'start_time_SHA09',
    'end_time_SHA09',
    'start_time_SHA10',
    'end_time_SHA10',
    'start_time_LGA01',
    'end_time_LGA01',
    'start_time_LGA02',
    'end_time_LGA02',
    'start_time_LGA03',
    'end_time_LGA03',
    'start_time_LGA04',
    'end_time_LGA04',
    'start_time_LGA05',
    'end_time_LGA05',
    'start_time_LGA06',
    'end_time_LGA06',
    'start_time_LGA07',
    'end_time_LGA07',
    'start_time_LGA08',
    'end_time_LGA08',
    'start_time_LGA09',
    'end_time_LGA09',
    'start_time_LGA10',
    'end_time_LGA10',
    'start_time_LGA11',
    'end_time_LGA11',
    'start_time_LGA12',
    'end_time_LGA12',
    'start_time_LGA13',
    'end_time_LGA13',
    'start_time_LGA14',
    'end_time_LGA14',
    'start_time_PR01',
    'end_time_PR01',
    'start_time_PR02',
    'end_time_PR02',
    'start_time_PR03',
    'end_time_PR03',
    'lga_post_treatment_1_date',
    'lga_post_treatment_2_date',
    'lga_post_treatment_3_date',
    'lga_post_treatment_4_date',
    'treatment_1_start_time',
    'treatment_2_start_time',
    'treatment_3_start_time',
    'treatment_4_start_time',
    'start_time_TREATMENT01',
    'end_time_TREATMENT01', 
    'start_time_TREATMENT02',
    'end_time_TREATMENT02',
    'start_time_TREATMENT03',
    'end_time_TREATMENT03',
    'start_time_TREATMENT04',
    'end_time_TREATMENT04',
    'start_time_PRESHOCK',
    'end_time_PRESHOCK',
    'start_time_SHOCK_V1',
    'end_time_SHOCK_V1',
    'start_time_SHOCK_V2',
    'end_time_SHOCK_V2',
    'start_time_SHOCK_V3',
    'end_time_SHOCK_V3',
    'start_time_SHA11',
    'end_time_SHA11',
    'start_time_LGA15',
    'end_time_LGA15',
    'start_time_LGA16',
    'end_time_LGA16',
    'start_time_LGA17',
    'end_time_LGA17',
    'start_time_LGA18',
    'end_time_LGA18',
    'start_time_LGA19',
    'end_time_LGA19',
    'start_time_LGA20',
    'end_time_LGA20',
    'start_time_LGA21',
    'end_time_LGA21',
    'start_time_LGA22',
    'end_time_LGA22',
    'start_time_LGA23',
    'end_time_LGA23',
    'start_time_LGA24',
    'end_time_LGA24',
    'start_time_LGA25',
    'end_time_LGA25',
    'start_time_LGA26',
    'end_time_LGA26',
    'start_time_LGA27',
    'end_time_LGA27',
    'start_time_LGA28',
    'end_time_LGA28',
    'treatment_1_date',
    'treatment_2_date',
    'treatment_3_date',
    'treatment_4_date',
    'von_frey_1_date',
    'von_frey_2_date',
    'tail_immersion_1_date',
    'tail_immersion_2_date',
    'tail_immersion_3_date',
    'irritability_1_date',
    'irritability_2_date',
    'female_swab_1_date',
    'female_swab_2_date',
    'female_swab_3_date',
    'lga_pre_treatment_1_date',
    'lga_pre_treatment_2_date',
    'lga_pre_treatment_3_date',
    'lga_pre_treatment_4_date',
    'lga_15_date',
    'lga_16_date',
    'lga_17_date',
    'lga_18_date',
    'lga_19_date',
    'lga_20_date',
    'handled_by',
    'surgeon',
    'surgery_assist',
    'recatheter_surgeon',
    'brevital_technicians',
    'irr_bsl_scorer_1',
    'irr_bsl_scorer_2',
    'irr_bsl_scorer_3',
    'irr_drug_scorer_1',
    'irr_drug_scorer_2',
    'irr_drug_scorer_3',
    'irritability_1_technicians',
    'irritability_2_technicians',
    'von_frey_1_technicians',
    'von_frey_2_technicians',
    'tail_immersion_1_technicians',
    'tail_immersion_2_technicians',
    'tail_immersion_3_technicians',
    'female_swab_1_technicians',
    'female_swab_2_technicians',
    'female_swab_3_technicians',
    'technician_urine_1',
    'technician_urine_2',
    'technician_feces_1',
    'technician_feces_2',
    'technician_feces_3',
    'technician_feces_4',
    'technician_weight_1',
    'technician_weight_2',
    'technician_weight_3',
    'technician_weight_4',
    'technician_weight_5',
    'technician_weight_6',
    'technician_weight_7',
    'technician_weight_8',
    'technician_weight_9',
    'technician_weight_10',
    'technician_weight_11',
    'technician_weight_12',
]

In [131]:
len(ordered_cols)

697

In [135]:
test

,rfid,rat,cohort,sex,drug_group,experiment_group,dissection_group,brevital,last_good_session,exit_code,...,technician_weight_3,technician_weight_4,technician_weight_5,technician_weight_6,technician_weight_7,technician_weight_8,technician_weight_9,technician_weight_10,technician_weight_11,technician_weight_12
0,933000120124701,F1,1,Female,Cocaine,Drug,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,933000120124703,F2,1,Female,Cocaine,Drug,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,933000120117342,F3,1,Female,Cocaine,Drug,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,933000120117333,F4,1,Female,Cocaine,Drug,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,933000120117347,F5,1,Female,Cocaine,Drug,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2376,933000320625139,M1883,18,Male,Oxycodone,Naive,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2377,933000320624877,M1884,18,Male,Oxycodone,Naive,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2378,933000320625141,M1885,18,Male,Oxycodone,Naive,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2379,933000320624891,M1886,18,Male,Oxycodone,Naive,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [132]:
test = test[ordered_cols]

In [134]:
test.to_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Unstacked/datastream_combined_tables_no_dates_ordered.csv')

In [ ]:
# "MTF134EC15HSOXYLGA" : [M1569 ......]